In [1]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine


In [ ]:
# Specify the path to your JSON configuration file
config_file_path = "/Users/gabisanches/Desktop/CIS9440 - Data Warehouse/Homework/Homework_GabrieleSanches_CIS9440/scripts/config.json"

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)


CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'carcrash'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)
# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)


car_crash_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.shape)
    # sind I have only one csv, I am doing to do the following instructions
    car_crash_df = df.copy()

car_crash.csv


In [ ]:
#printcolumns in the data set
print(car_crash_df.columns)

In [ ]:
car_crash_df.head()

In [ ]:
#dropping columns that are not needed
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_3'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code_3'])
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_4'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code_4'])
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_5'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code_5'])

In [ ]:
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_2'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code2'])

In [ ]:
car_crash_df = car_crash_df.drop(columns=['location'])

In [ ]:
car_crash_df = car_crash_df.drop(columns=['cross_street_name'])

In [ ]:
car_crash_df.head()

In [ ]:
#rename columns
car_crash_df.rename(columns={'contributing_factor_vehicle_1': 'contributing_factor_vehicle'}, inplace=True)
car_crash_df.rename(columns={'vehicle_type_code1': 'vehicle_type_code'}, inplace=True)

In [ ]:
#check column names after initial cleaning and transformation
print(car_crash_df.columns)

In [ ]:
#drop na - drop rows that do not have data in the "on_street_name" column
#car_crash_df['on_street_name'].dropna(inplace=True)
car_crash_df = car_crash_df[car_crash_df['on_street_name'].notna()]

car_crash_df.head()

In [ ]:
#reformat to int format 
car_crash_df['number_of_persons_injured'] = car_crash_df['number_of_persons_injured'].astype('Int64')
car_crash_df['number_of_persons_killed'] = car_crash_df['number_of_persons_killed'].astype('Int64')
car_crash_df['number_of_pedestrians_injured'] = car_crash_df['number_of_pedestrians_injured'].astype('Int64')
car_crash_df['number_of_pedestrians_killed'] = car_crash_df['number_of_pedestrians_killed'].astype('Int64')
car_crash_df['number_of_cyclist_injured'] = car_crash_df['number_of_cyclist_injured'].astype('Int64')
car_crash_df['number_of_cyclist_killed'] = car_crash_df['number_of_cyclist_killed'].astype('Int64')
car_crash_df['number_of_motorist_injured'] = car_crash_df['number_of_motorist_injured'].astype('Int64')
car_crash_df['number_of_motorist_killed'] = car_crash_df['number_of_motorist_killed'].astype('Int64')

car_crash_df.head()

In [ ]:
print(car_crash_df.columns)

In [ ]:
#Creating vehicle_type dimension

# Get unique values from the 'vehicle_type_code' column
unique_vehicle_types = car_crash_df['vehicle_type_code'].unique()

# Create DataFrame for the dimension table
dim_vehicle_type = pd.DataFrame(unique_vehicle_types, columns=['vehicle_type_code'])

# Add primary key column
dim_vehicle_type['vehicle_id'] = range(1, len(dim_vehicle_type) + 1)

# Rearrange columns if needed
dim_vehicle_type = dim_vehicle_type[['vehicle_id', 'vehicle_type_code']]

# Print dimension table
print(dim_vehicle_type)


In [ ]:
#Creating contributing_factors dimension

# Get unique values from the 'contributing_factor_vehicle' column
unique_contributing_factors = car_crash_df['contributing_factor_vehicle'].dropna().unique()

# Create DataFrame for the dimension table
dim_contributing_factors = pd.DataFrame(unique_contributing_factors, columns=['contributing_factor'])

# Add primary key column
dim_contributing_factors['contributingfactor_id'] = range(1, len(dim_contributing_factors) + 1)

# Rearrange columns if needed
dim_contributing_factors = dim_contributing_factors[['contributingfactor_id', 'contributing_factor']]

# Print dimension table
print(dim_contributing_factors)

In [ ]:
#Creating location dimension

# Extract relevant columns for dim_location
dim_location_df = car_crash_df[['borough', 'latitude', 'longitude', 'zip_code', 'on_street_name', 'off_street_name']]

# Drop duplicate rows
dim_location_df.drop_duplicates(inplace=True)

# Reset index
dim_location_df.reset_index(drop=True, inplace=True)

# Add location_id as primary key
dim_location_df['location_id'] = dim_location_df.index + 1

# Rearrange columns
dim_location_df = dim_location_df[['location_id', 'borough', 'latitude', 'longitude', 'zip_code', 'on_street_name', 'off_street_name']]

# Print dimension table
print(dim_location_df)

In [ ]:
# Define a function to calculate the week of the month
def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

# Extract relevant date-related columns from your dataset
start_date = car_crash_df['crash_date'].min()
end_date = car_crash_df['crash_date'].max()

# Create a DataFrame for the date dimension
date_dimension = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='H')})

# Extract attributes
date_dimension['year_number'] = date_dimension['date'].dt.year
date_dimension['quarter_number'] = date_dimension['date'].dt.quarter
date_dimension['month_number'] = date_dimension['date'].dt.month
date_dimension['month_name'] = date_dimension['date'].dt.strftime('%B')
date_dimension['day_number'] = date_dimension['date'].dt.day
date_dimension['day_name'] = date_dimension['date'].dt.strftime('%A')
date_dimension['hour_number'] = date_dimension['date'].dt.hour
date_dimension['date_iso_format'] = date_dimension['date'].apply(lambda x: x.isoformat())
date_dimension['date_id'] = date_dimension['date'].dt.strftime('%Y%m%d%H')

# Add week of the month and week of the year
date_dimension['week_of_month'] = date_dimension['date'].apply(week_of_month)
date_dimension['week_of_year'] = date_dimension['date'].dt.strftime('%U')

# Add crash_time column
#date_dimension['time'] = car_crash_df['crash_time']

# Define the order of columns in the date dimension
new_order = ['date_id', 'date_iso_format','year_number','quarter_number','month_number','month_name','day_number','day_name', 'hour_number','week_of_month','week_of_year']
date_dimension = date_dimension[new_order]

# Print the date dimension
print(date_dimension.head(25))

In [ ]:
# Step 1: Merge dimension tables with the original dataset

# Merge with dim_location to add location_id
car_crash_df = car_crash_df.merge(dim_location, on=['borough', 'latitude', 'longitude', 'zip_code', 'on_street_name', 'off_street_name', 'cross_street_name'], how='left')

# Merge with dim_vehicle_type to add vehicle_id
car_crash_df = car_crash_df.merge(dim_vehicle_type, on='vehicle_type_code', how='left')

# Merge with dim_contributing_factors to add contributingfactor_id
car_crash_df = car_crash_df.merge(dim_contributing_factors, on='contributing_factor_vehicle', how='left')

# Merge with dim_date to add date_id
car_crash_df = car_crash_df.merge(dim_date, on='crash_date', how='left')

# Step 2: Rename columns
new_column_names = {
    'collision_id': 'fact_id',
    'number_of_persons_injured': 'number_of_persons_injured',
    'number_of_persons_killed': 'number_of_persons_killed',
    'number_of_pedestrians_injured': 'number_of_pedestrians_injured',
    'number_of_pedestrians_killed': 'number_of_pedestrians_killed',
    'number_of_cyclist_injured': 'number_of_cyclist_injured',
    'number_of_cyclist_killed': 'number_of_cyclist_killed',
    'number_of_motorist_injured': 'number_of_motorist_injured',
    'number_of_motorist_killed': 'number_of_motorist_killed',
    'number_of_accidents': 'number_of_accidents',
    'location_id': 'location_id',
    'vehicle_id': 'vehicle_id',
    'contributingfactor_id': 'contributingfactor_id'
    # Add other columns and their corresponding names here
}

car_crash_df = car_crash_df.rename(columns=new_column_names)

# Step 3: Reorder columns
new_order = ['fact_id', 'number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed', 'number_of_accidents', 'location_id', 'date_id', 'vehicle_id', 'contributingfactor_id']

car_crash_df = car_crash_df[new_order]
car_crash_df.head()
